In [3]:
#importing libraries
import pandas as pd

In [5]:
df =  pd.read_csv('dataset/synthetic_logs.csv')
df

,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert
...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert


In [6]:
df.source.unique()

<StringArray>
[      'ModernCRM', 'AnalyticsEngine',        'ModernHR',   'BillingSystem',
   'ThirdPartyAPI',       'LegacyCRM']
Length: 6, dtype: str

In [7]:
df.target_label.unique()

<StringArray>
[        'HTTP Status',      'Critical Error',      'Security Alert',
               'Error', 'System Notification',      'Resource Usage',
         'User Action',      'Workflow Error', 'Deprecation Warning']
Length: 9, dtype: str

In [11]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
import numpy as np

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for log messages
embeddings = model.encode(df['log_message'].tolist(), show_progress_bar=True)
embeddings[:2]

# # Display results
# print(f"Number of clusters: {len(set(clustering.labels_)) - (1 if -1 in clustering.labels_ else 0)}")
# print(f"Number of noise points: {list(clustering.labels_).count(-1)}")
# print(df[['log_message', 'cluster']].head(10))

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 132.87it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Batches: 100%|██████████| 76/76 [01:35<00:00,  1.25s/it]


array([[-1.02939621e-01,  3.35459262e-02, -2.20260601e-02,
         1.55105593e-03, -9.86920949e-03, -1.78956255e-01,
        -6.34409785e-02, -6.01761676e-02,  2.81108841e-02,
         5.99620454e-02, -1.72618646e-02,  1.43367471e-03,
        -1.49560004e-01,  3.15285777e-03, -5.66031076e-02,
         2.71685384e-02, -1.49890380e-02, -3.54037546e-02,
        -3.62936258e-02, -1.45410234e-02, -5.61493542e-03,
         8.75539109e-02,  4.55120429e-02,  2.50963680e-02,
         1.00187706e-02,  1.24267014e-02, -1.39923617e-01,
         7.68696442e-02,  3.14095207e-02, -4.15248703e-03,
         4.36902680e-02,  1.71250310e-02, -8.00951496e-02,
         5.74005917e-02,  1.89091805e-02,  8.55262280e-02,
         3.96399237e-02, -1.34371832e-01, -1.44364929e-03,
         3.06707853e-03,  1.76854029e-01,  4.44886740e-03,
        -1.69275180e-02,  2.24266723e-02, -4.35049348e-02,
         6.09028572e-03, -9.98169184e-03, -6.23973049e-02,
         1.07372776e-02, -6.04895549e-03, -7.14660808e-0

In [16]:
# Apply DBSCAN clustering
clustering = DBSCAN(eps=0.2, min_samples=1, metric='cosine').fit(embeddings)
clusters = clustering.fit_predict(embeddings)

#add cluster labels to the original dataframe
df['cluster'] = clustering.labels_  
df

,timestamp,source,log_message,target_label,complexity,cluster
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0
...,...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert,0
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert,7
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert,0
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert,1


In [15]:
df[df.cluster==1]

,timestamp,source,log_message,target_label,complexity,cluster
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,1
6,3/1/2025 19:14,ModernHR,Shard 6 replication task ended in failure,Error,bert,1
10,8/9/2025 18:58,ModernCRM,Email server encountered a sending fault,Error,bert,1
11,6/15/2025 11:44,ModernHR,Critical system unit error: unit ID Component55,Critical Error,bert,1
...,...,...,...,...,...,...
2398,3/31/2025 4:11,ModernHR,Potential vulnerability exploit detected from ...,Security Alert,bert,1
2402,3/13/2025 9:44,BillingSystem,Replication error occurred for shard 10,Error,bert,1
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert,1
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert,1


In [17]:
# Sort clusters by number of records
cluster_counts = df['cluster'].value_counts().sort_values(ascending=False)

# Filter clusters with more than 10 records
large_clusters = cluster_counts[cluster_counts > 10].index

# Print 5 log messages from each large cluster
for cluster_id in large_clusters:
    print(f"\n--- Cluster {cluster_id} ({cluster_counts[cluster_id]} records) ---")
    print(df[df['cluster'] == cluster_id]['log_message'].head(5).values)    


--- Cluster 0 (1017 records) ---
<StringArray>
[           'nova.osapi_compute.wsgi.server [req-b9718cd8-f65e-49cc-8349-6cf7122af137 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2675118',
 'nova.osapi_compute.wsgi.server [req-4895c258-b2f8-488f-a2a3-4fae63982e48 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" HTTP status code -  200 len: 211 time: 0.0968180',
             'nova.osapi_compute.wsgi.server [req-ee8bc8ba-9265-4280-9215-dbe000a41209 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" RCODE  200 len: 1874 time: 0.2280791',
       'nova.osapi_compute.wsgi.server [req-f0bffbc3-5ab0-4916-91c1-0a61dd7d4ec2 113d3a99c3da401fbd62cc2caa5b96d2 54fadb4

In [26]:
import re
def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out)": "User action",
        r"Backup (started|ended) at .*": "System notification",
        r"backup completed successfully": "System notification",    
        r"System updated to version .*" : "System notification",
        r"file .* uploaded successfully by user .*": "system notification",
        r"disk cleanup completed successfully": "System notification",
        r"system reboot initialized by user .*": "System notification",
        r"account with ID .* created by .*":"user action",
        r"account with ID .* deleted by .*":"user action",
        r"failed login attempt for user .* from IP .*":"security event",
        r"unauthorized access attempt detected from IP .*":"security event",
        r"malware detected in file .* uploaded by user .*":"security event",
    }

    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message, re.IGNORECASE):
            return label
        return None

In [27]:
classify_with_regex("User User123 logged in")

'User action'

In [32]:
df['regex_label'] = df['log_message'].apply(classify_with_regex)
df[df.regex_label.notnull()]

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
27,9/24/2025 19:57,ThirdPartyAPI,User User685 logged out.,User Action,regex,11,User action
57,9/14/2025 3:03,AnalyticsEngine,User User395 logged in.,User Action,regex,11,User action
85,3/13/2025 2:11,ModernHR,User User225 logged in.,User Action,regex,11,User action
88,3/8/2025 19:04,AnalyticsEngine,User User494 logged out.,User Action,regex,11,User action
126,11/22/2025 21:09,ThirdPartyAPI,User User900 logged in.,User Action,regex,11,User action
...,...,...,...,...,...,...,...
2207,10/4/2025 8:06,ModernCRM,User User495 logged in.,User Action,regex,11,User action
2263,2/27/2025 14:40,AnalyticsEngine,User User429 logged out.,User Action,regex,11,User action
2275,3/13/2025 17:17,AnalyticsEngine,User User755 logged out.,User Action,regex,11,User action
2323,12/1/2025 18:17,ThirdPartyAPI,User User882 logged out.,User Action,regex,11,User action


In [33]:
df_non_regex = df[df['regex_label'].isnull()].copy()
df_non_regex

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,NaN
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,NaN
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,NaN
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,NaN
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,NaN
...,...,...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert,0,NaN
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert,7,NaN
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert,0,NaN
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert,1,NaN


In [34]:
# Get target labels with 5 or fewer rows in df_non_regex
label_counts = df_non_regex['target_label'].value_counts()
small_labels = label_counts[label_counts <= 5]
print(small_labels)

target_label
Workflow Error         4
Deprecation Warning    3
Name: count, dtype: int64


In [35]:
df_non_legacy = df_non_regex[df_non_regex.source != 'LegacyCRM']
df_non_legacy.source.unique()

<StringArray>
['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem', 'ThirdPartyAPI']
Length: 5, dtype: str

In [36]:
# Generate embeddings for log messages
embeddings = model.encode(df['log_message'].tolist(), show_progress_bar=True)
embeddings[:2]

Batches: 100%|██████████| 76/76 [01:48<00:00,  1.43s/it]


array([[-1.02939621e-01,  3.35459262e-02, -2.20260601e-02,
         1.55105593e-03, -9.86920949e-03, -1.78956255e-01,
        -6.34409785e-02, -6.01761676e-02,  2.81108841e-02,
         5.99620454e-02, -1.72618646e-02,  1.43367471e-03,
        -1.49560004e-01,  3.15285777e-03, -5.66031076e-02,
         2.71685384e-02, -1.49890380e-02, -3.54037546e-02,
        -3.62936258e-02, -1.45410234e-02, -5.61493542e-03,
         8.75539109e-02,  4.55120429e-02,  2.50963680e-02,
         1.00187706e-02,  1.24267014e-02, -1.39923617e-01,
         7.68696442e-02,  3.14095207e-02, -4.15248703e-03,
         4.36902680e-02,  1.71250310e-02, -8.00951496e-02,
         5.74005917e-02,  1.89091805e-02,  8.55262280e-02,
         3.96399237e-02, -1.34371832e-01, -1.44364929e-03,
         3.06707853e-03,  1.76854029e-01,  4.44886740e-03,
        -1.69275180e-02,  2.24266723e-02, -4.35049348e-02,
         6.09028572e-03, -9.98169184e-03, -6.23973049e-02,
         1.07372776e-02, -6.04895549e-03, -7.14660808e-0

In [38]:
# Get indices of df_non_legacy rows
indices = df_non_legacy.index

# Use only embeddings for df_non_legacy rows
X = embeddings[indices]
y = df_non_legacy['target_label']

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

                     precision    recall  f1-score   support

     Critical Error       0.94      1.00      0.97        46
              Error       1.00      0.94      0.97        52
        HTTP Status       1.00      1.00      1.00       294
     Resource Usage       1.00      1.00      1.00        50
     Security Alert       1.00      1.00      1.00       120
System Notification       1.00      1.00      1.00       115
        User Action       1.00      1.00      1.00        14

           accuracy                           1.00       691
          macro avg       0.99      0.99      0.99       691
       weighted avg       1.00      1.00      1.00       691



In [54]:
import joblib
joblib.dump(clf, 'models/log_classifier.joblib')

['models/log_classifier.joblib']